In [ ]:
# Import Dependencies
import requests
import json
from bs4 import BeautifulSoup
import time
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from pandas import to_datetime

In [ ]:
# Read CSV File
filt_data_df = pd.read_csv("tables/main_table.csv")
filt_data_df

In [ ]:
filt_data_df['publishDate'].value_counts

In [ ]:
# Cleaned the published date column to bring all dates to one format
filt_data_df["publishedDate"]= pd.to_datetime(filt_data_df['publishDate'],errors = 'coerce')
filt_data_df

In [ ]:
filt_data_df=filt_data_df.dropna()

In [ ]:
filt_data_df['yearPublished'] = pd.DatetimeIndex(filt_data_df['publishedDate']).year
filt_data_df

In [ ]:
# Dropped irregular date format column after cleaning
filt_data_df.drop(columns="publishDate")

In [ ]:
# Saving dataframe as csv
filt_data_df.to_csv("tables/year.csv", index= False)

In [ ]:
# Filtering the data for latest 5 yrs
filt_data_df.sort_values(by="yearPublished", ascending=False)

In [ ]:
#df = df[(df['closing_price'] >= 99) & (df['closing_price'] <= 101)]
filt_data_df=filt_data_df[(filt_data_df['yearPublished']>=2011)&(filt_data_df['yearPublished']<=2020)].reset_index(inplace=False)

In [47]:
filt_data_df.count()

title             11868
rating            11868
description       11868
language          11868
isbn              11868
genres            11868
pages             11868
publishDate       11868
publisher         11868
numRatings        11868
ratingsByStars    11868
likedPercent      11868
main author       11868
publishedDate     11868
yearPublished     11868
dtype: int64

In [ ]:
filt_data_df = filt_data_df.drop(columns=["index","Unnamed: 0"])

In [46]:
# Saving dataframe as csv
filt_data_df.to_csv("tables/main_table.csv", index= False)

In [50]:
# Make list of authors
author_list = filt_data_df['main author'].tolist()
author_list1 = author_list[:5000]
len(author_list1)

5000

In [ ]:
# Base url
base_url = "https://en.wikipedia.org/wiki/"
author_dict = {}
for author_name in author_list1:
    # create url along with author name
    url = base_url + author_name
    # Retrieve page with the requests module
    response = requests.get(url)
    try:
    # Create BeautifulSoup object; parse with 'html.parser'
        soup = BeautifulSoup(response.text, 'html.parser')
        author_dict[author_name]= soup.find('span', class_="bday").text
    except AttributeError:
        author_dict[author_name]= "none"

   
   
author_dict  